For this computer lab, we'll be using the IRIS dataset. Initially, we'll only look at a subset of it, and perform linear regression on two features of a given class.

# 1. Loading the data

### 1.1  Import the necessary modules

We'll use these three different modules.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### 1.2  Read the dataset from a .csv file

Load the [IRIS dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set) using Pandas. The method `read_csv()` returns a `DataFrame` object containing the data found in the provided .csv file.

In [ ]:
dataset = pd.read_csv("IRIS.csv")

In [ ]:
type(dataset)

### 1.3  Analyze the dataset

This dataset is comprised of morphologic data from three different species of the Iris flowers: Setosa, Virginica and Versicolor.

<table style="width:100%">
  <tr>
    <th> <center>Iris Setosa</center> </th>
    <th> <center>Iris Virginica</center> </th> 
    <th> <center>Iris Versicolor</center> </th>
  </tr>
  <tr>
    <td>![Iris Setosa](http://www.hih-gruppen.se/bilder/irisgalleri/skagglos/IMsibLB/setosa.jpg)</td>
    <td>![Iris Virginica](https://upload.wikimedia.org/wikipedia/commons/9/9f/Iris_virginica.jpg)</td>
    <td>![Iris Versicolor](https://upload.wikimedia.org/wikipedia/commons/4/41/Iris_versicolor_3.jpg)</td>
  </tr>
</table>

The lenght and width of both the petals and the sepals of each flower, together with its corresponding species were measured and stored in this dataset. Sepals and petals are both parts of a flower. Sepals are the outermost part of the whorl and the petals are the innermost part.
![](http://terpconnect.umd.edu/~petersd/666/html/iris_with_labels.jpg)

Let's take a look at what's inside the dataset now. The attribute `shape` of `DataFrame` objects returns the dimensions of the data inside it.

In [ ]:
dataset.shape

So this dataset has 150 rows and 5 columns. It's easy to infer that this means 150 flowers were collected, and 5 different features were registered for each one. But we can also take a closer look at them, using the method `head()`, which returns the first 5 rows by default (you can also pass a parameter to it, which specifies a different amount of rows to be shown).

In [ ]:
dataset.head()

Here we can see the header names for each column, together with the first rows, confirming that the species and morphologic measurements for each flower were collected. We can extract individual columns of this `DataFrame` by indexing using their names, for instance:

In [ ]:
dataset["sepal_length"]

Additionally, we can check which species are present in the dataset using the `unique` method,

In [ ]:
print(dataset["species"].unique())

where we see that only these three species are present in this dataset, as expected.

We can also learn more about the data types of each column with the method `info`.

In [ ]:
dataset.info()

Here we see that the first four columns' elements are floating point numbers, and the last column's elements are objects (in this case, strings).

### 1.4  Extract the desired data

For this initial task, we are only interested in the setosa species. This corresponds to all the rows which have the column 'species' equal to the string 'setosa'. In order to extract these rows, we use [logical indexing in Pandas](https://pandas.pydata.org/pandas-docs/stable/indexing.html#boolean-indexing).

In [ ]:
# This returns a boolean series, which we can then use to index our DataFrame object
extract_rule = dataset['species']=='setosa'

In [ ]:
extract_rule

In [ ]:
# We use the boolean series to index the DataFrame object
setosa_dataset = dataset[extract_rule]
setosa_dataset

Furthermore, we want to investigate the relationship between two features of this species, the 'sepal_length' and 'sepal_width'. To extract these, we [index the `DataFrame` using the name of the columns](https://pandas.pydata.org/pandas-docs/stable/indexing.html#selection-by-label)  we want.

In [ ]:
x = setosa_dataset['sepal_length'].values
y = setosa_dataset['sepal_width'].values

Note that the attribute `values` in a `DataFrame` object returns a numpy array.

In [ ]:
type(x)

Now we can use matplotlib to plot all the examples in a 2D plane, where each dimension is one of the features described earlier.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x,y)
ax.set_xlabel('sepal length')
ax.set_ylabel('sepal width');

It seems like the relation between these features could be approximated using a linear function, such as 
$f(x) = w\cdot x + b$. Let's try finding the parameters $w$ and $b$ that would make the best approximation.

### 1.5  Guess the values of w and b

We'll start with some educated guesses. To make this more convenient, we'll first define a function to plot a scatter plot of the provided data, together with a straight line with parameters specified by the user.

In [ ]:
# Define a function to plot the data and a parameterized line
def plot_data_and_line(w, b, x, y, ax, line_color='r', line_label=''):
    
    # Create points lying on the line
    xline = np.unique(x)
    yline = w*xline + b

    # Plot both the line and the points from the dataset
    ax.scatter(x,y, color='C0')
    ax.plot(xline, yline, color=line_color, label=line_label)
    ax.set_xlabel('sepal length')
    ax.set_ylabel('sepal width') 

Additionally, another way of evaluating the quality of our approximation is to compute the MSE (mean squared error) between the true y features in the dataset and our predictions. So that we can use this value as well to guide our guesses, create a function to compute it (first, it might be beneficial to write down the analytical expression for it).

In [ ]:
# Create a function to compute the MSE


Now we can try different values of $w$ and $b$ and see how the resulting linear approximation looks like, compared to the scatter plot of our data. Using both the plot and the MSE, try searching for values of $w$ and $b$ that yield a good approximation.

In [ ]:
# Guess the values for w and b
w = 2
b = -6.5

# Plot your guess
plt.close('all')
fig, ax = plt.subplots()
plot_data_and_line(w, b, x, y, ax);

# Compute MSE of the guess
y_guess = w*x+b
print("MSE of your guess:", MSE(y,y_guess))

---

# 2. Training a model for linear regression (8m)

Now, instead of trying to find the parameters that give the best approximation by trial and error, we'll use Keras' framework to build and optimize a linear regressor neural network. Actually, we'll be using Keras and Tensorflow.

Tensorflow is what is called the 'backend' of Keras in this case, taking care of the matrix computations and parallelization necessary to speed up the code. Keras, on the other hand, is the Python module we'll use to interface with Tensorflow, using higher-level abstractions. That is, instead of thinking about the low-level details of tensors and the actual computations involved, we'll be thinking about neurons and network architectures, optimizers, etc.

### 2.1  Importing the necessary modules

We start by loading the necessary modules from Keras.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

### 2.2  Create the Keras model

The problem of linear regression that we've been tackling so far can be seen as training a neural network consisting of only one neuron, with one weight and one bias. This model can be easily specified and trained in Keras, as we'll show ahead.

In Keras, [models can be of two different types](https://keras.io/models/about-keras-models/#about-keras-models). For this task, it's enough to use the simpler one, the [`sequential`](https://keras.io/models/sequential/) module. The following code initializes such a model and then calls its `add` method in order to add the first (and only) layer. Finally, this calls the [`compile`](https://keras.io/models/sequential/#compile) method, in which we configure the learning process by specifying the loss and optimizer to be used.

In [ ]:
model = Sequential()
model.add(Dense(1, input_dim=1, kernel_initializer='zeros', bias_initializer='zeros'))
model.compile(loss='mean_squared_error', optimizer='SGD')

### 2.3  Optimize the model

Now that we have specified the model and how we want to train it, calling the [`fit`](https://keras.io/models/sequential/#fit) method starts the training process.

In [ ]:
model.fit(x,y,epochs=50);

Note the final MSE obtained (~0.066). Compare it to the one obtained using the guessed parameters.

### 2.4  Extract optimal parameters

Although the `fit` method does show us the final obtained MSE, it does not display the optimized parameters. To obtain those, we use the `get_weights` method of the layer we're interested in (in this case, the only layer), which returns the weights of the layer as a numpy array.

In [ ]:
optimal_w_np = model.layers[0].get_weights()[0]
optimal_b_np = model.layers[0].get_weights()[1]

type(optimal_w_np)

Right now it's more convenient to have these as floating point numbers instead, so we'll go ahead and convert them.

In [ ]:
optimal_w = float(optimal_w_np)
optimal_b = float(optimal_b_np)
print("w: %.3f" % optimal_w)
print("b: %.3f" % optimal_b)

Compare these optimized parameters with the ones you guessed before.

### 2.5  Compare optimal and guessed values

Finally, it's also beneficial to compare the guessed parameters with the optimized ones graphically, by showing both of the predicted lines in the same plot.

In [ ]:
plt.close('all')
fig, ax = plt.subplots()
plot_data_and_line(w, b, x, y, ax, 'r', 'guess')
plot_data_and_line(optimal_w, optimal_b, x, y, ax, 'b', 'optimal')
ax.legend();

---

# Bonus: Visualizing the optimization (10m)

To clearly understand what's going on when we call the Keras `fit` function, it's helpful to illustrate the optimization trajectory. That is, we would like to plot the level curves of the loss function in the parameter space, together with the values of `w` and `b` at each time step of the iteration.

First, we import the `History` callback class, along with Adam and SGD optimizers.

In [ ]:
from keras.callbacks import Callback
from keras.optimizers import Adam, SGD

Create a callback to save the weights at each iteration.

In [ ]:
# Create a callback to save the weights at each iteration
class saveWeightsCallback(Callback):
    def on_train_begin(self, logs=None):
        self.w = [float(self.model.layers[0].get_weights()[0])]
        self.b = [float(self.model.layers[0].get_weights()[1])]
    
    def on_epoch_end(self, epoch, logs=None):
        self.w.append(float(self.model.layers[0].get_weights()[0]))
        self.b.append(float(self.model.layers[0].get_weights()[1]))

Now we train the model, passing the newly created callback as argument to the `fit` method.

In [ ]:
# Create a model to be optimized with SGD
model = Sequential()
model.add(Dense(1, input_dim=1, kernel_initializer='zeros', bias_initializer='zeros'))
model.compile(loss='mean_squared_error', optimizer=SGD(lr=0.01, momentum=0))

# Train it
print('Training...')
history = saveWeightsCallback()
model.fit(x,y,epochs=100, callbacks=[history], verbose=0, batch_size=32);
print('Done!')

We also need to compute the MSE in a grid of points, so that we can plot level curves with `matplotlib` `contour` method.

In [ ]:
wmin = min(history.w)
wmax = max(history.w)
bmin = min(history.b)
bmax = max(history.b)

# Create grid of values
n = 400
w_range = np.linspace(min(0,wmin),max(2,wmax),n)
b_range = np.linspace(min(-1,bmin), max(1,bmax),n)
w_grid, b_grid = np.meshgrid(w_range, b_range)

# Compute MSE at each one
MSE_grid = np.ndarray((n,n))
for i in range(n):
    for j in range(n):
        w = w_grid[i,j]
        b = b_grid[i,j]
        y_ = w*x + b
        MSE_grid[i,j] = MSE(y_,y)

Now we're ready to plot. 

However, before doing that, can you guess how the level curves for this loss should look like? Will they be circles, ellipses, something else? Will they be aligned with the axes? Do we have one or many global optima?

What about the optimization trajectory? How should it look like? What do you expect to see?

In [ ]:
# Plot the MSE level curves 
plt.close('all')
fig, ax = plt.subplots(figsize=(15,10))
p = ax.contour(w_grid, b_grid, MSE_grid, np.linspace(0, 40, 100), cmap='hot')

# Plot the optimization trajectory
ax.plot(history.w, history.b, '-ro')

# Plot the global optimum
optimal = np.polyfit(x,y,1)
ax.plot(optimal[0], optimal[1], 'bx')

# Axis' labels, colorbar
fig.colorbar(p, ax=ax, ticks=np.linspace(0,40,9))
ax.set_xlabel('w')
ax.set_ylabel('b');

Try different batch sizes, learning rates, number of epochs, or even a different optimizer, like Adam.